In [1]:
# -*- coding: utf-8 -*-
# --------------------------------------------------
#
# CRF3.ipynb
#
# Token features:
#     (a) token (surface form)
#     (b) is_first           : is token at the beginning of the sentence?
#     (c) is_last            : is token at the end of the sentence?
#     (d) is_capitalized     : does token start with a capital letter? 
#     (e) is_all_capitalized : is all letters of the token capitalized?
#     (f) is_capitals_inside : is there any capitalized letter inside the token?
#     (g) is_numeric         : is token numeric?
#     (h) is_numeric_inside  : is numeric characters inside the token?
#     (i) is_alphanumeric    : is token alphanumeric?
#     (j) prefix-1           : first letter of the token
#     (k) suffix-1           : last letter of the token
#
# Written by cetinsamet -*- cetin.samet@metu.edu.tr
# May, 2019
# --------------------------------------------------

In [3]:
from sklearn.model_selection import RandomizedSearchCV
from seqeval.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn_crfsuite import CRF
from tqdm import tqdm
import pickle
import scipy

In [6]:
def readFile(filepath):

    text     = []
    sentence = []

    with open(filepath, 'r') as infile:
        for line in infile:
            word, _, _, _ = line.strip().split('\t')

            if word == '<S>':
                text.append(sentence)
                sentence = []
                continue

            sentence.append(line.strip())

    return text

In [7]:
trainText = readFile('data/train.txt')
validText = readFile('data/valid.txt')
testText  = readFile('data/test.txt')

In [8]:
def getFeature(token, token_index, sentence):

    feature = {'token'             : token,
               'is_first'          : token_index == 0,
               'is_last'           : token_index == len(sentence) - 1,
               
               'is_capitalized'    : token[0].upper() == token[0],
               'is_all_capitalized': token.upper() == token,
               'is_capitals_inside': token[1:].lower() != token[1:],
               'is_numeric'        : token.isdigit(),
               'is_numeric_inside' : any([c.isdigit() for c in token]),
               'is_alphanumeric'   : token.isalnum(),
               
               'prefix-1'          : token[0],
               'suffix-1'          : token[-1],
              }
    
    return feature

In [9]:
trainFeatures = []
trainLabels   = []

for sentence in tqdm(trainText):

    features = []
    labels   = []
    for i, token in enumerate(sentence):
        word, _, _, label = token.split('\t')
        
        features.append(getFeature(word, i, sentence))
        labels.append(label)
    
    trainFeatures.append(features)
    trainLabels.append(labels)

100%|██████████| 25736/25736 [00:01<00:00, 14455.85it/s]


In [10]:
validFeatures = []
validLabels   = []

for sentence in tqdm(validText):
    
    features = []
    labels   = []
    for i, token in enumerate(sentence):
        word, _, _, label = token.split('\t')
        
        features.append(getFeature(word, i, sentence))
        labels.append(label)
    
    validFeatures.append(features)
    validLabels.append(labels)

100%|██████████| 6435/6435 [00:00<00:00, 11991.79it/s]


In [12]:
testFeatures = []
testLabels   = []

for sentence in tqdm(testText):
    
    features = []
    labels   = []
    for i, token in enumerate(sentence):
        word, _, _, label = token.split('\t')
        
        features.append(getFeature(word, i, sentence))
        labels.append(label)
    
    testFeatures.append(features)
    testLabels.append(labels)

100%|██████████| 3328/3328 [00:00<00:00, 13611.18it/s]


In [14]:
trainvalFeatures = trainFeatures + validFeatures
trainvalLabels   = trainLabels   + validLabels

In [15]:
# define fixed parameters and parameters to search
crf = CRF(  algorithm='lbfgs',
            max_iterations=100,
            all_possible_transitions=True,
            verbose=True)

params_space = {'c1': scipy.stats.expon(scale=0.5),
                'c2': scipy.stats.expon(scale=0.05)}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=30,
                        random_state=123,
                        scoring=f1_scorer)

rs.fit(trainvalFeatures, trainvalLabels)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/cetinsamet/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 64.4min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 150.6min finished
loading training data to CRFsuite: 100%|██████████| 32171/32171 [00:16<00:00, 1967.21it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 76564
Seconds required: 1.857

L-BFGS optimization
c1: 0.048313
c2: 0.028432
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=2.02  loss=703749.30 active=76541 feature_norm=1.00
Iter 2   time=3.06  loss=404027.12 active=76342 feature_norm=4.22
Iter 3   time=1.05  loss=340909.33 active=75441 feature_norm=3.68
Iter 4   time=3.03  loss=298730.70 active=76019 feature_norm=2.62
Iter 5   time=1.05  loss=266143.23 active=76324 feature_norm=3.37
Iter 6   time=1.01  loss=248596.11 active=76374 feature_norm=3.35
Iter 7   time=1.12  loss=230413.19 active=76320 feature_norm=3.82
Iter 8   time=1.08  loss=217632.93 active=76471 feature_norm=4.10
Iter 9   time=1.07  loss=188347.20 active=76524 feature_norm=5.61
Iter 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=None,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error...ne,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1b00d149b0>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1b00d14c88>},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn',
          scoring=make_scorer(flat_f1_score, average=weighted), verbose=1)

In [16]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.04831323984330459, 'c2': 0.028431668831944812}
best CV score: 0.9648608250803564
model size: 1.53M


In [17]:
crf = rs.best_estimator_
crf

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.04831323984330459, c2=0.028431668831944812,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True)

In [19]:
#crf.fit(trainvalFeatures, trainvalLabels)

In [20]:
# SAVE CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf3.pickle', 'wb') as outfile:
    pickle.dump(crf, outfile, pickle.HIGHEST_PROTOCOL)
    print("model is saved.")

model is saved.


In [21]:
# LOAD CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf3.pickle', 'rb') as infile:
    crf = pickle.load(infile)

In [22]:
trainvalPredLabels = crf.predict(trainvalFeatures)

print("### TRAINVAL CLASSIFICATION REPORT ###\n")
print(classification_report(trainvalLabels, trainvalPredLabels))

### TRAINVAL CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

    LOCATION       0.98      0.99      0.98      9409
        DATE       0.96      0.97      0.96      3103
       MONEY       0.98      0.97      0.98       594
ORGANIZATION       0.97      0.97      0.97      9034
      PERSON       0.98      0.98      0.98     14476
        TIME       0.99      1.00      1.00       175
     PERCENT       1.00      1.00      1.00       617

   micro avg       0.98      0.98      0.98     37408
   macro avg       0.98      0.98      0.98     37408



In [23]:
testPredLabels  = crf.predict(testFeatures)

print("### TEST CLASSIFICATION REPORT ###\n")
print(classification_report(testLabels, testPredLabels))

### TEST CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

      PERSON       0.89      0.79      0.84      1594
    LOCATION       0.88      0.82      0.85      1091
        DATE       0.88      0.84      0.86       364
ORGANIZATION       0.79      0.77      0.78       862
       MONEY       0.84      0.72      0.78       113
     PERCENT       0.99      0.95      0.97       107
        TIME       0.83      0.65      0.73        23

   micro avg       0.87      0.80      0.83      4154
   macro avg       0.87      0.80      0.83      4154

